# Graph Event System Examples

**All examples print actual event output!**

In [ ]:
from lfx.debug.event_recorder import record_graph_with_events
from lfx.debug.events import GraphMutationEvent
from lfx.graph import Graph
import json

## Example 1: Dependency Events

The most important event - when run_predecessors and run_map change.

In [ ]:
# Direct API call to see dependency events
graph = Graph()

async def print_deps(event: GraphMutationEvent):
    if event.event_type == 'dependency_added':
        print(f"\n{event.timing.upper()} Event (Step {event.step}):")
        print(f"  Vertex: {event.changes['vertex']}")
        print(f"  Predecessor: {event.changes['predecessor']}")
        
        if event.timing == 'after':
            print(f"\n  State AFTER mutation:")
            print(json.dumps(event.state_after, indent=4))
            print(f"\n  ✅ run_predecessors changed: {event.changes['run_predecessors_changed']}")
            print(f"  ✅ run_map changed: {event.changes['run_map_changed']}")

graph.register_observer(print_deps)

print('Calling graph.add_dynamic_dependency("ComponentA", "ComponentB")...')
await graph.add_dynamic_dependency('ComponentA', 'ComponentB')

print('\n💡 Both structures updated atomically - impossible to desync!')

## Example 2: Queue Events

See how the execution queue changes.

In [ ]:
graph2 = Graph()

async def print_queue(event: GraphMutationEvent):
    if 'queue' in event.event_type and event.timing == 'after':
        before_size = event.changes.get('queue_size_before', 0)
        after_size = event.changes.get('queue_size_after', 0)
        
        print(f"Step {event.step}: {event.event_type}")
        print(f"  Queue: {before_size} → {after_size} items")
        
        if event.changes.get('added'):
            print(f"  Added: {event.changes['added']}")
        if event.changes.get('removed'):
            print(f"  Removed: {event.changes['removed']}")

graph2.register_observer(print_queue)

await graph2.extend_run_queue(['v1', 'v2'])
await graph2.extend_run_queue(['v3'])
await graph2.get_next_in_queue()

## Example 3: Vertex State Events

When vertices change state (ACTIVE/INACTIVE).

In [ ]:
from lfx import components as cp
from lfx.graph.vertex.base import VertexStates

# Create a graph with vertices
inp = cp.ChatInput()
out = cp.ChatOutput().set(input_value=inp.message_response)
graph3 = Graph(inp, out)
graph3.prepare()

async def print_states(event: GraphMutationEvent):
    if event.event_type == 'vertex_marked' and event.timing == 'after':
        vertex = event.vertex_id.split('-')[0] if event.vertex_id else '?'
        old = event.changes.get('old_state')
        new = event.changes.get('new_state')
        print(f"Step {event.step}: {vertex} state {old} → {new}")

graph3.register_observer(print_states)

# Manually mark a vertex to trigger event
vertex_id = graph3.vertices[0].id
print(f'Marking {vertex_id.split("-")[0]} as INACTIVE...')
await graph3.mark_vertex(vertex_id, VertexStates.INACTIVE)

## Example 4: Complete Event Structure

See all fields in an event.

In [ ]:
# Record a simple flow
recording = await record_graph_with_events(Graph(inp, out), 'Demo')

print(f'Captured {len(recording.events)} events')

# Show complete structure of first event
if recording.events:
    print('\nComplete event structure:')
    print(json.dumps(recording.events[0].to_dict(), indent=2, default=str))

## Summary

**Key Event Types:**
- `dependency_added`: run_predecessors & run_map updated
- `vertex_marked`: State changes (ACTIVE/INACTIVE)
- `queue_extended/dequeued`: Queue operations

**Every event contains:**
- state_before & state_after
- changes (what changed)
- graph_snapshot (complete context)
- timing (before/after)

**Benefits:**
- Zero overhead without observers
- Complete state history
- Impossible to desync structures
- Extensible for custom tracking